### Hugging Face의 transformers 라이브러리의 기능을 활용하여 모델 학습 수행

In [1]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [2]:
!pip install pytorch-lightning==1.6.1
!pip install transformers==4.28.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.5/582.5 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.4/840.4 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 54.7 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.1
    Uninstalling transformers-4.38.1:
      Successfully uninstalled transformers-4.38.1


In [3]:
!pip install evaluate datasets -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.4 MB/s eta 0:00:00


파인튜닝을 위한 말뭉치
- GLUE (General Language Understanding Evaluation) Benchmark :
인간의 언어 능력을 인공지능이 얼마나 따라왔는지 정량적 성능지표를 만든 것이라고 할 수 있다.

- MRPC(Microsoft Research Paraphrase Corpus): 이 데이터셋은 두 문장의 의미적 유사도를 사람이 정한 데이터 셋이다. The Microsoft Research Paraphrase Corpus의 줄임말로써, 문장 쌍과 그에 대한 라벨로 구성되어 있다.
- QQP(Quora Question Paraphrases): 이 데이터셋은 네이버 지식인과 같은 곳으로 부터 질문들의 유사도를 측정해 둔 것으로, 동일한 문장인지 아닌지에 대해 binary로 표현되어있다.

bert-base-uncased는 Google에서 개발한 BERT(Bidirectional Encoder Representations from Transformers) 모델의 한 변형입니다. BERT는 2018년에 소개되었으며, 광범위한 자연어 처리(NLP) 작업에서 혁신적인 성능을 보여주었습니다. bert-base-uncased 모델은 다음과 같은 특징을 가지고 있습니다:
- 모델 사이즈: bert-base-uncased는 약 1억 1천만 개의 파라미터를 가진 모델로, BERT의 더 큰 변형인 bert-large에 비해 상대적으로 작은 편입니다. bert-base는 12개의 Transformer 블록(layers), 768개의 hidden size, 그리고 12개의 self-attention heads를 가집니다.
- 언케이스드(Uncased): 이 모델은 텍스트의 대소문자를 구분하지 않습니다. 즉, 입력 데이터는 소문자로 변환되며, 이는 다양한 언어 사용 환경에서의 일관성을 유지하기 위함입니다. 예를 들어, "The"와 "the"는 모델에 의해 동일하게 처리됩니다.

In [4]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

# GLUE 벤치마크의 MRPC 태스크 데이터셋을 로드
dataset = load_dataset("glue", "mrpc")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [5]:
# GLUE 벤치마크의 MRPC 태스크 데이터셋을 로드
dataset = load_dataset("glue", "mrpc")

# 데이터셋의 구조 확인
print(dataset)

# 트레이닝 세트의 첫 번째 예시를 출력해보기
print(dataset['train'][0])

# 데이터셋의 칼럼 이름들 확인
print(dataset['train'].column_names)


DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})
{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .', 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .', 'label': 1, 'idx': 0}
['sentence1', 'sentence2', 'label', 'idx']


In [6]:
# 트레이닝 세트의 첫 번째 예시를 출력해보기
print(dataset['test'][0])

# 데이터셋의 칼럼 이름들 확인
print(dataset['test'].column_names)


{'sentence1': "PCCW 's chief operating officer , Mike Butcher , and Alex Arena , the chief financial officer , will report directly to Mr So .", 'sentence2': 'Current Chief Operating Officer Mike Butcher and Group Chief Financial Officer Alex Arena will report to So .', 'label': 1, 'idx': 0}
['sentence1', 'sentence2', 'label', 'idx']


In [7]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

# GLUE 벤치마크의 MRPC 태스크 데이터셋을 로드
dataset = load_dataset("glue", "mrpc")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["sentence1"], examples["sentence2"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

학습인자 설정
- TrainingArguments: 학습 과정에서 사용할 다양한 설정을 정의합니다.
- output_dir: 학습 결과(모델 체크포인트, 설정 등)를 저장할 디렉토리입니다.
- num_train_epochs: 전체 데이터셋에 대한 학습을 몇 번 반복할지 지정합니다.
- per_device_train_batch_size와 per_device_eval_batch_size: 각각 학습과 평가 시 배치 크기를 지정합니다.
- warmup_steps: 학습률 스케줄러의 예열 단계 수를 지정합니다. 이는 모델 학습 초기에 학습률을 점진적으로 증가시키는 데 사용됩니다.
- weight_decay: 가중치 감소를 위한 매개변수입니다. 오버피팅을 방지하는 데 도움이 됩니다.
- evaluation_strategy: 평가 전략을 지정합니다. 여기서는 'epoch'으로 설정되어 있어, 각 에포크가 끝날 때마다 평가가 수행됩니다.
- logging_dir: 학습 로그를 저장할 디렉토리입니다.

트레이너 생성 및 학습 실행
- Trainer: 모델 학습을 위한 클래스입니다. 학습을 관리하고 실행하는 데 필요한 모든 정보(모델, 데이터셋, 학습 인자 등)를 받습니다.
- train_dataset과 eval_dataset: 각각 학습과 평가에 사용할 데이터셋을 지정합니다. 이전 단계에서 토크나이징 처리된 데이터셋을 사용합니다.
- trainer.train(): 설정된 인자와 데이터셋을 사용해 모델 학습을 시작합니다.

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)

trainer.train()


BERT와 같은 모델을 파인튜닝한 후 저장하는 방법에는 주로 두 가지가 있습니다:

save_pretrained를 사용하여 전체 모델을 저장하는 방법
- 설명: save_pretrained는 Hugging Face의 Transformers 라이브러리에 있는 메소드로, 모델의 가중치와 함께 토크나이저 설정을 포함하여 전체 모델을 저장하는 데 사용됩니다. 이 방법은 모델을 쉽게 재사용하고 배포할 수 있게 해줍니다.
- 사용 방법: 모델과 토크나이저를 파인튜닝합니다.
파인튜닝된 모델과 토크나이저를 save_pretrained 메소드를 사용하여 저장합니다.
  - model.save_pretrained('./my_bert_model')
  - tokenizer.save_pretrained('./my_bert_model')
  - model = BertForSequenceClassification.from_pretrained('./my_saved_model')
  - tokenizer = BertTokenizer.from_pretrained('./my_saved_model')

체크포인트(checkpoint)만 저장하는 방법
- 설명: 체크포인트 저장은 주로 훈련 과정 중 특정 시점의 모델 상태(가중치)만 저장하는 것을 말합니다. 이 방법은 긴 훈련 과정에서 중간 결과를 저장하고, 필요시 이어서 훈련할 수 있게 해줍니다.
- 훈련 과정에서 정기적으로 또는 특정 조건을 만족할 때 모델의 state_dict를 저장합니다.
  - torch.save(model.state_dict(), './checkpoint.pth')
  - model.load_state_dict(torch.load('./checkpoint.pth'))
  - 토크나이저는 일반적으로 해당 사전학습 모델에 맞는 것을 다시 가져와서 사용할 수 있습니다.

모델 저장
- model_path 변수에 모델과 토크나이저를 저장할 경로를 지정합니다. 여기서는 "paircl_bert_model"이라는 이름의 디렉토리로 설정되어 있습니다.
- model.save_pretrained(model_path)를 호출하여 학습된 모델을 model_path에 지정된 경로에 저장합니다. 이 메소드는 모델의 가중치를 포함한 구성 파일을 해당 경로에 저장합니다. 이렇게 저장된 모델은 나중에 from_pretrained 메소드를 사용하여 쉽게 로드할 수 있습니다.

토크나이저 저장
- tokenizer.save_pretrained(model_path)를 사용하여 토크나이저를 model_path에 지정된 경로에 저장합니다. 이 과정은 토크나이저의 설정과 어휘집(vocab) 파일을 디스크에 저장합니다.
- 모델과 토크나이저를 같은 경로에 저장함으로써, 추후 모델을 재사용할 때 토크나이저와 모델을 함께 쉽게 로드할 수 있습니다. 이는 모델의 입력 데이터를 올바르게 전처리하는 데 필수적인 토크나이저의 정보를 유지하기 위함입니다.

In [ ]:
# 모델 저장
model_path = "/gdrive/MyDrive/nlpbook/paircl_bert_model"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('paircl_bert_model/tokenizer_config.json',
 'paircl_bert_model/special_tokens_map.json',
 'paircl_bert_model/vocab.txt',
 'paircl_bert_model/added_tokens.json')

In [ ]:
!ls

logs  paircl_bert_model  results  sample_data


In [ ]:
!ls ./paircl_bert_model

config.json  pytorch_model.bin	special_tokens_map.json  tokenizer_config.json	vocab.txt


In [ ]:
# mkdir과 함께 -p 옵션을 사용하여 상위 디렉터리를 만들 수 있습니다
!mkdir "/gdrive/MyDrive/kdt_231026/m7_nlp응용/data/paircl_bert_model"

In [ ]:
!cp -r ./paircl_bert_model /gdrive/MyDrive/kdt_231026/m7_nlp응용/data/